In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import statsmodels.api as sm

## 5.3.1 The Validation Set Approach

In [2]:
auto = pd.read_csv('../data/auto.csv', na_values=['?'])
auto.dropna(axis=0, inplace=True)

auto['horsepower2'] = np.power(auto['horsepower'], 2)
auto['horsepower3'] = np.power(auto['horsepower'], 3)

In [3]:
from sklearn.model_selection import train_test_split

X = auto.loc[:, ['horsepower', 'horsepower2', 'horsepower3']]
y = auto.mpg

X_train, X_cv, y_train, y_cv = train_test_split(X, y, train_size=0.5, random_state=1)

In [4]:
from sklearn.linear_model import LinearRegression

sk_lm = LinearRegression(fit_intercept=True)
sk_lm2 = LinearRegression(fit_intercept=True)
sk_lm3 = LinearRegression(fit_intercept=True)

sk_lm.fit(X_train['horsepower'].values.reshape(-1, 1), y_train)
sk_lm2.fit(X_train.loc[:, ['horsepower', 'horsepower2']], y_train)
sk_lm3.fit(X_train.loc[:, ['horsepower', 'horsepower2', 'horsepower3']], y_train)

sk_lm_predict = sk_lm.predict(X_cv['horsepower'].values.reshape(-1, 1))
sk_lm2_predict = sk_lm2.predict(X_cv.loc[:, ['horsepower', 'horsepower2']])
sk_lm3_predict = sk_lm3.predict(X_cv.loc[:, ['horsepower', 'horsepower2', 'horsepower3']])

In [5]:
from sklearn.metrics import mean_squared_error

print(f'Linear: {mean_squared_error(y_cv, sk_lm_predict)}')
print(f'Quadratic: {mean_squared_error(y_cv, sk_lm2_predict)}')
print(f'Cubic: {mean_squared_error(y_cv, sk_lm3_predict)}')

Linear: 24.80212062059356
Quadratic: 18.84829260327566
Cubic: 18.80511135860503


## 5.3.2 Leave-One-Out Cross-Validation

In [6]:
auto['horsepower4'] = np.power(auto['horsepower'], 4)
auto['horsepower5'] = np.power(auto['horsepower'], 5)
auto['horsepower6'] = np.power(auto['horsepower'], 6)

In [7]:
X, y = auto.loc[:, ['horsepower', 'horsepower2', 'horsepower3', 'horsepower4', 'horsepower5', 'horsepower6']], auto.mpg

In [8]:
from sklearn.model_selection import LeaveOneOut

loocv = LeaveOneOut()
loocv.get_n_splits(X)

loocv_lm = LinearRegression(fit_intercept=True)

loocv_mse = []
for train_idx, cv_idx in loocv.split(X):
    X_train, X_cv = X.iloc[train_idx], X.iloc[cv_idx]
    y_train, y_cv = y.iloc[train_idx], y.iloc[cv_idx]
    
    loocv_lm.fit(X_train.loc[:, 'horsepower'].values.reshape(-1, 1), y_train)
    loocv_mse.append(mean_squared_error(y_cv, loocv_lm.predict(X_cv.loc[:, 'horsepower'].values.reshape(-1, 1))))

np.array(loocv_mse).mean()

24.231513517929233

In [9]:
# polynomial features

loocv_outputs = {}

for i in range(0, 6):
    loocv_mse = []
    
    lm = LinearRegression(fit_intercept=True)
    
    for train_idx, cv_idx in loocv.split(X):
        X_train, X_cv = X.iloc[train_idx], X.iloc[cv_idx]
        y_train, y_cv = y.iloc[train_idx], y.iloc[cv_idx]
        
        if i == 0:
            X_TRAIN = X_train.iloc[:, 0].values.reshape(-1, 1)
            X_CV = X_cv.iloc[:, 0].values.reshape(-1, 1)
        else:
            X_TRAIN = X_train.iloc[:, 0:i+1]
            X_CV = X_cv.iloc[:, 0:i+1]
        
        lm.fit(X_TRAIN, y_train)
        prediction = lm.predict(X_CV)
        loocv_mse.append(mean_squared_error(y_cv, prediction))
        
    loocv_outputs[f'lm-degree:{i+1}-MSE'] = np.array(loocv_mse).mean()

In [10]:
loocv_outputs

{'lm-degree:1-MSE': 24.231513517929233,
 'lm-degree:2-MSE': 19.24821312448965,
 'lm-degree:3-MSE': 19.33498406402896,
 'lm-degree:4-MSE': 19.424430310426498,
 'lm-degree:5-MSE': 19.0332121143411,
 'lm-degree:6-MSE': 18.97806840155051}

## 5.3.3 k-Fold Cross-Validation

In [11]:
X, y = auto.loc[:, ['horsepower', 'horsepower2', 'horsepower3', 'horsepower4', 'horsepower5', 'horsepower6']], auto.mpg

In [12]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=17)
kf.get_n_splits(X)

kf_lm = LinearRegression(fit_intercept=True)

kf_mse = []
for train_idx, cv_idx in kf.split(X):
    X_train, X_cv = X.iloc[train_idx], X.iloc[cv_idx]
    y_train, y_cv = y.iloc[train_idx], y.iloc[cv_idx]
    
    kf_lm.fit(X_train.loc[:, 'horsepower'].values.reshape(-1, 1), y_train)
    prediction = kf_lm.predict(X_cv.loc[:, 'horsepower'].values.reshape(-1, 1))
    
    kf_mse.append(mean_squared_error(y_cv, prediction))

np.array(kf_mse).mean()

24.182335895569228

In [13]:
# polynomial features

kf_outputs = {}

for i in range(0, 6):
    kf_mse = []
    
    lm = LinearRegression(fit_intercept=True)
    
    for train_idx, cv_idx in kf.split(X):
        X_train, X_cv = X.iloc[train_idx], X.iloc[cv_idx]
        y_train, y_cv = y.iloc[train_idx], y.iloc[cv_idx]
        
        if i == 0:
            X_TRAIN = X_train.iloc[:, 0].values.reshape(-1, 1)
            X_CV = X_cv.iloc[:, 0].values.reshape(-1, 1)
        else:
            X_TRAIN = X_train.iloc[:, 0:i+1]
            X_CV = X_cv.iloc[:, 0:i+1]
        
        lm.fit(X_TRAIN, y_train)
        prediction = lm.predict(X_CV)
        kf_mse.append(mean_squared_error(y_cv, prediction))
        
    kf_outputs[f'lm-degree:{i+1}-MSE'] = np.array(kf_mse).mean()

In [14]:
kf_outputs

{'lm-degree:1-MSE': 24.182335895569228,
 'lm-degree:2-MSE': 19.137584705467884,
 'lm-degree:3-MSE': 19.155247225776385,
 'lm-degree:4-MSE': 19.250162771368508,
 'lm-degree:5-MSE': 18.897086745494697,
 'lm-degree:6-MSE': 18.81696248200453}

## 5.3.4 The Bootstrap

In [15]:
portfolio = pd.read_csv('../data/portfolio.csv')
portfolio.head()

,X,Y
0,-0.895251,-0.234924
1,-1.562454,-0.885176
2,-0.417090,0.271888
3,1.044356,-0.734198
4,-0.315568,0.841983


### Estimating the Accuracy of a Statistic of Interest

In [16]:
def alpha_fn(data, index):
    X = data['X'][index[0]: index[1]]
    y = data['Y'][index[0]: index[1]]
    return ((np.var(y) - np.cov(X, y)[0][1])/(np.var(X) + np.var(y) - 2 * np.cov(X, y)[0][1]))

In [17]:
alpha_fn(portfolio, [0, 100])

0.5766511516104116

In [18]:
from sklearn.utils import resample

portfolio_samp = resample(portfolio, replace=True, n_samples=100, random_state=7)
portfolio_samp

,X,Y
47,1.171830,1.729831
68,-0.881701,-1.540685
25,-0.685376,-0.457616
67,1.302851,1.104666
83,-0.600486,-0.420941
...,...,...
76,0.716797,0.602337
67,1.302851,1.104666
14,0.294016,0.628589
20,-0.645448,-1.412431


In [19]:
alpha_fn(portfolio_samp, [0, 100])

0.5117744709121624

In [20]:
bootstrap_alpha = []

for i in range(1000):
    portfolio_samp = resample(portfolio, replace=True, n_samples=100)
    bootstrap_alpha.append(alpha_fn(portfolio_samp, [0, 100]))
    
bs_alpha = np.array(bootstrap_alpha)
print('Bootstrap alpha: ', bs_alpha.mean())
print('SE: ', bs_alpha.std())

Bootstrap alpha:  0.5807311300821817
SE:  0.0944201601875738


### Estimating the Accuracy of a Linear Regression Model

In [21]:
def boot_fn(data, index):
    lm = LinearRegression(fit_intercept=True)
    lm.fit(data['horsepower'][index[0]:index[1]].values.reshape(-1, 1),
           data['mpg'][index[0]:index[1]])
    return lm.intercept_, lm.coef_

In [22]:
boot_fn(auto, [0, 392])

(39.935861021170524, array([-0.15784473]))

In [23]:
bs_intercept = []
bs_coef = []

for i in range(1000):
    bs_intercept.append(boot_fn(resample(auto, replace=True, n_samples=392), [0, 392])[0])
    bs_coef.append(boot_fn(resample(auto, replace=True, n_samples=392), [0, 392])[1][0])

result_table = pd.DataFrame({'Bootstrap Estimate': [np.array(bs_intercept).mean(), np.array(bs_coef).mean()],
                             'Bootstrap SE': [np.array(bs_intercept).std(), np.array(bs_coef).std()]},
                           index=['intercept', 'coefficient'])

result_table

,Bootstrap Estimate,Bootstrap SE
intercept,39.969382,0.881983
coefficient,-0.158017,0.007197


In [24]:
lm = sm.OLS(auto.mpg, sm.add_constant(auto.horsepower)).fit()
lm.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,39.9359,0.717,55.660,0.000,38.525,41.347
horsepower,-0.1578,0.006,-24.489,0.000,-0.171,-0.145


In [25]:
def boot_fn2(data, index):
    lm = LinearRegression(fit_intercept=True)
    lm.fit(data[['horsepower', 'horsepower2']][index[0]:index[1]],
           data['mpg'][index[0]:index[1]])
    return lm.intercept_, lm.coef_

In [26]:
bs2_intercept = []
bs2_coef1 = []
bs2_coef2 = []

for i in range(1000):
    bs2_intercept.append(boot_fn2(resample(auto, replace=True, n_samples=392), [0, 392])[0])
    bs2_coef1.append(boot_fn2(resample(auto, replace=True, n_samples=392), [0, 392])[1][0])
    bs2_coef2.append(boot_fn2(resample(auto, replace=True, n_samples=392), [0, 392])[1][1])
    
result_table = pd.DataFrame({
                     'Bootstrap Estimate': [np.array(bs2_intercept).mean(),
                                            np.array(bs2_coef1).mean(),
                                            np.array(bs2_coef2).mean()],
                     'Bootstrap SE': [np.array(bs2_intercept).std(), 
                                      np.array(bs2_coef1).std(),
                                      np.array(bs2_coef2).std()]},
                     index=['intercept', 'coefficient1', 'coefficient2'])
result_table

,Bootstrap Estimate,Bootstrap SE
intercept,56.892466,2.065909
coefficient1,-0.467487,0.033874
coefficient2,0.001242,0.000119


In [27]:
lm2 = sm.OLS(auto.mpg, sm.add_constant(auto[['horsepower', 'horsepower2']])).fit()
lm2.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,56.9001,1.800,31.604,0.000,53.360,60.440
horsepower,-0.4662,0.031,-14.978,0.000,-0.527,-0.405
horsepower2,0.0012,0.000,10.080,0.000,0.001,0.001
